## 당뇨병 여부 판단
- 예측컬럼: Outcome (0 정상, 1 당뇨) 당뇨병일 확률 예측
- 평가지표: roc-auc
- 제출파일명: result.csv (1개컬럼, 컬럼명 pred)

## 라이브러리 및 데이터 불러오기

In [1]:
# import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score

In [2]:
# 1. 데이터 로드
train=pd.read_csv("/kaggle/input/bigdatacertificationkr/diabetes_train.csv")
test=pd.read_csv("/kaggle/input/bigdatacertificationkr/diabetes_test.csv")

print(f'"Train shape: {train.shape}')
print(f'"Test shape: {test.shape}')
print('-----------------------------')
print('데이터 정보 확인')
print(train.info())
print('-----------------------------')
print('데이터 예시 확인')
print(test.head(10))
print('-----------------------------')
print("결측치 확인:")
print(train.isnull().sum())

"Train shape: (576, 10)
"Test shape: (192, 9)
-----------------------------
데이터 정보 확인
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        576 non-null    int64  
 1   Pregnancies               576 non-null    int64  
 2   Glucose                   576 non-null    int64  
 3   BloodPressure             576 non-null    int64  
 4   SkinThickness             576 non-null    int64  
 5   Insulin                   576 non-null    int64  
 6   BMI                       576 non-null    float64
 7   DiabetesPedigreeFunction  576 non-null    float64
 8   Age                       576 non-null    int64  
 9   Outcome                   576 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 45.1 KB
None
-----------------------------
데이터 예시 확인
    id  Pregnancies  Glucose  BloodPressure  SkinThic

In [3]:

# 2. 타겟 분리
X_train = train.drop('Outcome',axis=1)  # 타겟 제거
y_train = train['Outcome']  

print('y_train예시 ')
print(y_train.head(10))
print('------------------------')
print('X_train예시 ')
print(X_train.head(10))
print('------------------------')
print('타겟 분포')
print(y_train.value_counts())

y_train예시 
0    0
1    1
2    1
3    1
4    1
5    1
6    1
7    0
8    0
9    1
Name: Outcome, dtype: int64
------------------------
X_train예시 
    id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0  377            1       87             60             37       75  37.2   
1  370            3      173             82             48      465  38.4   
2  408            8      197             74              0        0  25.9   
3  535            4      132              0              0        0  32.9   
4  691           13      158            114              0        0  42.3   
5  319            6      194             78              0        0  23.5   
6  292            2      128             78             37      182  43.3   
7  118            4       97             60             23        0  28.2   
8  765            5      121             72             23      112  26.2   
9  646            1      167             74             17      144  23.4   

   Diab

In [4]:
# 3.결측치 처리
# 수치형: 중앙값으로 처리(이상치에 더 강하다)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
print(f'numeric_cols:{numeric_cols}')

for col in numeric_cols:
    median_val = X_train[col].median()

    # X_train이랑 test에 같은 값을 대입한다. 
    X_train[col] = X_train[col].fillna(median_val)
    test[col] = test[col].fillna(median_val)

# 범주형: 최빈값으로 처리
categorical_cols = X_train.select_dtypes(include=['object']).columns
print(f'categorical_cols:{categorical_cols}')

for col in categorical_cols:
    mode_val = X_train[col].mode()[0]

    # X_train이랑 test에 같은 값을 대입한다. 
    X_train[col] = X_train[col].fillna(mode_val)
    test[col] = test[col].fillna(mode_val)

print("결측치 확인:")
print(X_train.isnull().sum())
print(test.isnull().sum())

numeric_cols:Index(['id', 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
       'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
categorical_cols:Index([], dtype='object')
결측치 확인:
id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64
id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64


In [5]:
# 4. 인코딩
# 범주형 변수 레이블 인코딩
le_dict = {} # 인코더 저장소 
for col in categorical_cols:
    le = LabelEncoder()
    # train + test 합쳐서 fit (중요!)
    all_values = pd.concat([X_train[col], test[col]], axis=0)
    # 합친 것을 학습
    le.fit(all_values)

    # 학습 후, X_train,test에 변환 
    X_train[col] = le.transform(X_train[col])
    test[col] = le.transform(test[col])

    # 학습된 값을 le_dict에 저장. 나중에 새로운 데이터가 와도 같은 매핑을 사용하도록 
    le_dict[col] = le

# 잘 되었나 확인하기
print(f'X_train:{X_train.head(10)}')
print('-----------------------')
print(f'test:{test.head(10)}')

X_train:    id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0  377            1       87             60             37       75  37.2   
1  370            3      173             82             48      465  38.4   
2  408            8      197             74              0        0  25.9   
3  535            4      132              0              0        0  32.9   
4  691           13      158            114              0        0  42.3   
5  319            6      194             78              0        0  23.5   
6  292            2      128             78             37      182  43.3   
7  118            4       97             60             23        0  28.2   
8  765            5      121             72             23      112  26.2   
9  646            1      167             74             17      144  23.4   

   DiabetesPedigreeFunction  Age  
0                     0.509   22  
1                     2.137   25  
2                     1.191   39  
3   

In [6]:
# 5. 검증 데이터 분할
# 검증 데이터를 통해 미리 성능 확인!
# 80% 학습용, 20% 검증용 
# 분류문제=무조건 stratify사용!(같은 비율로 나눠지게 함)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print('X_train 데이터 분리 확인 ')
print(X_train.shape)
print(X_tr.shape)
print(X_val.shape)

print('y_train 데이터 분리 확인 ')
print(y_train.shape)
print(y_tr.shape)
print(y_val.shape)

X_train 데이터 분리 확인 
(576, 9)
(460, 9)
(116, 9)
y_train 데이터 분리 확인 
(576,)
(460,)
(116,)


In [7]:

# 6. 모델 학습 및 평가
# 모델 선택 
model = RandomForestClassifier(random_state=42, n_estimators=200)

# 모델 학습
model.fit(X_tr, y_tr)

# 검증 성능 확인
print(model.classes_)  # [0 1] 1(생존) 확률을 확인해보자. 

# [0 1] 1(생존) 확률을 확인해보자. 
val_pred_proba = model.predict_proba(X_val)[:, 1]
val_pred = model.predict(X_val)

# ROC-AUC,Accuracy는 1에 가까울수록 좋다. 85이상이면 좋다. 
print(f"ROC-AUC: {roc_auc_score(y_val, val_pred_proba)}")
print(f"Accuracy: {accuracy_score(y_val, val_pred)}")
# 맞는 label을 적용해야한다. 
print(f'F1_score:{f1_score(y_val, val_pred, pos_label=1)}')

[0 1]
ROC-AUC: 0.7963702963702964
Accuracy: 0.7068965517241379
F1_score:0.4848484848484849


In [8]:
# 7. 최종 예측 및 제출
# 전체 데이터로 학습
# 시간이 있다면 파라미터 조정(max_depth=10)
final_model = RandomForestClassifier(random_state=42, n_estimators=200)

# 모델 학습
final_model.fit(X_train, y_train)

# 테스트 예측 
# 당뇨병일 확률을 예측해야 한다. 
test_pred_proba = final_model.predict_proba(test)[:, 1]  

print("예측 결과 확인:")
print("클래스:", final_model.classes_)  # [0, 1]
print("처음 10개 예측:", test_pred_proba[:10])

# 제출 파일 생성
submit = pd.DataFrame({
    'pred': test_pred_proba  
})

submit.to_csv('submission-diabetes.csv', index=False)
print("제출 파일 생성 완료!")
print(submit.head(10))

예측 결과 확인:
클래스: [0 1]
처음 10개 예측: [0.615 0.735 0.005 0.85  0.305 0.38  0.68  0.015 0.365 0.45 ]
제출 파일 생성 완료!
    pred
0  0.615
1  0.735
2  0.005
3  0.850
4  0.305
5  0.380
6  0.680
7  0.015
8  0.365
9  0.450
